In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, dense_rank
from pyspark.sql.window import Window

# Criar sessão do Spark
spark = SparkSession.builder.appName("PySpark_Escola").getOrCreate()

# Criar dados para turmas
dados_turmas = pd.DataFrame({
    "id_turma": [1, 2, 3, 4],
    "nome_turma": ["Turma A", "Turma B", "Turma C", "Turma D"],
    "disciplina": ["Matemática", "História", "Física", "Química"],
    "ano": [2024, 2023, 2024, 2023],
    "num_alunos": [30, 25, 20, 28]
})

df_turmas = spark.createDataFrame(dados_turmas)

# Criar dados para alunos
dados_alunos = pd.DataFrame({
    "id_aluno": [1, 2, 3, 4, 5, 6],
    "nome": ["João Silva", "Maria Santos", "Carlos Nunes", "Ana Costa", "Rui Pereira", "Sara Mendes"],
    "turma": [1, 2, 3, 1, 2, 4],
    "idade": [16, 17, 15, 16, 18, 17],
    "desempenho": ["Bom", "Excelente", "Médio", "Bom", "Fraco", "Excelente"]
})

df_alunos = spark.createDataFrame(dados_alunos)

# Consultas SQL
print("1️⃣ Listar todas as turmas:")
df_turmas.show()

print("2️⃣ Listar apenas nome e disciplina das turmas:")
df_turmas.select("nome_turma", "disciplina").show()

print("3️⃣ Filtrar turmas do ano de 2024:")
df_turmas.filter(col("ano") == 2024).show()

print("4️⃣ Ordenar turmas por número de alunos (decrescente):")
df_turmas.orderBy(col("num_alunos").desc()).show()

print("5️⃣ Contar quantas turmas existem:")
print(df_turmas.count())

print("6️⃣ Listar todos os alunos:")
df_alunos.show()

print("7️⃣ Listar apenas nome e idade dos alunos:")
df_alunos.select("nome", "idade").show()

print("8️⃣ Filtrar alunos com idade superior a 16 anos:")
df_alunos.filter(col("idade") > 16).show()

print("9️⃣ Ordenar alunos por nome:")
df_alunos.orderBy("nome").show()

print("🔟 Contar quantos alunos existem:")
print(df_alunos.count())

print("1️⃣1️⃣ Listar alunos com desempenho 'Excelente':")
df_alunos.filter(col("desempenho") == "Excelente").show()

print("1️⃣2️⃣ Mostrar alunos da 'Turma A':")
df_alunos.filter(col("turma") == 1).show()

print("1️⃣3️⃣ Calcular a idade média dos alunos:")
df_alunos.selectExpr("AVG(idade) AS idade_media").show()

print("1️⃣4️⃣ Contar quantos alunos existem por turma:")
df_alunos.groupBy("turma").count().show()

print("1️⃣5️⃣ Listar alunos com categoria de idade:")
df_alunos.select("*", when(col("idade") <= 16, "Menor de idade").otherwise("Maior de idade").alias("categoria_idade")).show()

print("1️⃣6️⃣ Criar coluna 'aprovado' com base no desempenho:")
df_alunos.select("*", when(col("desempenho").isin(["Bom", "Excelente"]), "Sim").otherwise("Não").alias("aprovado")).show()

print("1️⃣7️⃣ Juntar dados de alunos e disciplinas via INNER JOIN:")
df_alunos.join(df_turmas, df_alunos.turma == df_turmas.id_turma).select("nome", "disciplina").show()

print("1️⃣8️⃣ Identificar turmas sem alunos:")
df_turmas.join(df_alunos, df_turmas.id_turma == df_alunos.turma, "left_anti").show()

print("1️⃣9️⃣ Criar ranking das turmas com mais alunos:")
windowSpec = Window.orderBy(col("num_alunos").desc())
df_turmas.withColumn("ranking", dense_rank().over(windowSpec)).show()

print("2️⃣0️⃣ Apresentar as duas turmas com mais alunos:")
df_turmas.orderBy(col("num_alunos").desc()).limit(2).show()

# Fechar sessão
spark.stop()


1️⃣ Listar todas as turmas:
+--------+----------+----------+----+----------+
|id_turma|nome_turma|disciplina| ano|num_alunos|
+--------+----------+----------+----+----------+
|       1|   Turma A|Matemática|2024|        30|
|       2|   Turma B|  História|2023|        25|
|       3|   Turma C|    Física|2024|        20|
|       4|   Turma D|   Química|2023|        28|
+--------+----------+----------+----+----------+

2️⃣ Listar apenas nome e disciplina das turmas:
+----------+----------+
|nome_turma|disciplina|
+----------+----------+
|   Turma A|Matemática|
|   Turma B|  História|
|   Turma C|    Física|
|   Turma D|   Química|
+----------+----------+

3️⃣ Filtrar turmas do ano de 2024:
+--------+----------+----------+----+----------+
|id_turma|nome_turma|disciplina| ano|num_alunos|
+--------+----------+----------+----+----------+
|       1|   Turma A|Matemática|2024|        30|
|       3|   Turma C|    Física|2024|        20|
+--------+----------+----------+----+----------+

4️⃣ Orde